In [7]:
#decoding: utf-8
#encoding: utf-8

#openCV transform C++ code to python
#python2.7  openCV3.4

#Sobel_locate

import numpy as np
import cv2 as cv
import os
import logging
import time 

def ini():  
    
    TM=time.strftime("%Y-%m-%d %H-%M-%S",time.localtime())
    LOG_FORMAT = "%(levelname)s -[:%(lineno)d]- %(message)s"
    logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
    logging.info('**************************Mason(%s)*******************'%(TM,))
    
def show(image,name):
    cv.imshow('%s'%name,image)
    cv.waitKey(0)
    logging.info('show image....')
    
def save_image(image,name):
    path_out=path+name+'.jpg'
    cv.imwrite(path_out,image)
    logging.info('save image:%s ,done'%name)
    

#####################
#高斯模糊，降噪
def Gauss(image):
    logging.info('Gauss...')
    image_Guass=cv.GaussianBlur(image, (5, 5), 1.5)
    return image_Guass

#图像灰度化, 三种方式：最大，平均，加权平均
def Gary(image):
    logging.info('Gary...')
    iamge_Gary=cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return iamge_Gary

#图像增强/边缘检测，sobel算子（选定水平边缘为检测对象）
def Enhencement(image):
    logging.info('Enhencement...')

    Sobel_x=0 # x horizontal，
    Sobel_y= cv.Sobel(image, cv.CV_16S,dx=1,dy=0,ksize=3)
    #dy=1 means vertical, ddepth=CV_16S;,ksize use default;ksize must be 1,3,5,7, what is default?=3
    image_Sobel=Sobel_y
    return image_Sobel

#二值化， 两种方式：全局阈值和 自适应阈值cv.adaptiveThreshold
def Binary(image):
    logging.info('Binary...')
    # for binary type, donot need :MaxvalMaxval=conf.getint('Binary', 'value')# set to be that value, 
    return_value,image_Binary=cv.threshold(image,127,255,cv.THRESH_BINARY)
    return image_Binary

#闭操作，区域连接
def Close(image):# should input binary image
    logging.info('Close...')
    kernel=cv.getStructuringElement(cv.MORPH_RECT,(20,20))  #must be int 方阵
    #func explaination   https://blog.csdn.net/qq_31186123/article/details/78770141
    image_Close=cv.morphologyEx(image,cv.MORPH_CLOSE,kernel)

    return image_Close


#计算边界/轮廓
#func explaination https://blog.csdn.net/hjxu2016/article/details/77833336
def Edge(path):#should input binary image
    logging.info('Edge...')
    #calculation
    image=cv.imread(path,cv.CV_8UC1) 
    binary,contours, hierarchy = cv.findContours(image,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE) 
    #in open CV3， findContours return 3 value; so my version is openCV3

    #draw boundary on the image
    image=cv.drawContours(image,contours,-1,(0,0,255),3)#-1:draw all edge;  3:line width；（0，0，255):RGB color scaler
    image_Edge=image
    return image_Edge,contours #outpur contours is a array, 


#不相连的轮廓求最小外界矩形,并添加到原图中
def draw_Min_Rect(image,contours):
    logging.info('draw_Min_Rect...')
    N= np.array(contours).shape[0]
    for i in range (0,N):
        min_=cv.minAreaRect(contours[i])
        box = cv.boxPoints(min_) 
        box = np.int0(box)
        cv.drawContours(image, [box], 0, (0,0,255), 3)
    #cv.imshow('0', image)
    #cv.waitKey(0)# the number of windows
    image_with_MinRect=image
    return image_with_MinRect



#角度筛选;角度判断与旋转。把倾斜角度大于阈值（如正负30度）的矩形舍弃
def Angle_filter(contours):
    logging.info('Angle_filter...')
    N= np.array(contours).shape[0]
    Box_update=[]
    contours_update=[]
    for i in range (0,N):
        Rect=cv.minAreaRect(contours[i])
        A=Rect[2]#anglle\
        
        if abs(A)<=20:
            contours_update.append(contours[i])
            Box_update.append(Rect)
    return contours_update,Box_update

#Area筛选box
def Rect_filter(contours):
    logging.info('Rect_filter...')
    N= np.array(contours).shape[0]
    contours_update=[]
    Box_update=[]
    for i in range (0,N):
        Rect=cv.minAreaRect(contours[i])
        Area_=Rect[1][0]*Rect[1][1]
        if Area_ >=500 and Area_<=12000 :
            print (Rect[1][0])/(Rect[1][1])

            #print Rect[2]
            if (Rect[1][0])/(Rect[1][1])>=3 and (Rect[1][0])/(Rect[1][1])<=5:
                print Area_
                contours_update.append(contours[i])
                Box_update.append(Rect)
    return contours_update,Box_update

#######################

#######################
#截取车牌box，
def CutImage(image_orig,Box_update):#Box_update is minAreaRect
    logging.info('CutImage...')
    points=cv.boxPoints(Box_update)
    left_min=min(points[0][0],points[1][0],points[2][0],points[3][0])
    right_max=max(points[0][0],points[1][0],points[2][0],points[3][0])
    up_max=max(points[0][1],points[1][1],points[2][1],points[3][1])
    down_min=min(points[0][1],points[1][1],points[2][1],points[3][1])
    CutImage=image_orig[down_min:up_max,left_min:right_max]

    return CutImage

#统一尺寸，classical 136*36， 但是LeNet 是默认36*36
def Unisize(CutImage):
    logging.info('Unisize...')
    width=conf.getint('Unisize','width')
    hight=conf.getint('Unisize','hight')
    Inter_Method=conf.get('Unisize',Inter_Method)
    cv.resize(CutImage,(width,hight),Inter_Method)
    return image_Unisize

def segment_box(image_Edge,contours,type_):
    logging.info('segment_box...')
    image_with_MinRect=draw_Min_Rect(image_Edge,contours)
    contours_Ang,box_ang=Angle_filter(contours)
    contours_Rect,box_rect=Rect_filter(contours_Ang)

    #save cpa
    for i in box_rect():
        CutImage(image,box_rect)
        image_Unisize=Unisize(CutImage)
        path_tmp=args["tmp_path"]+name
        cv.imwrite(path_tmp,image_Unisize)
    #SVM classification
    svm= SVM_(type_)
    for i in box_rect():
        CutImage(image,box_rect)
        image_Unisize=Unisize(CutImage)
        svm.predict(Unisize_image)
        if result ==Has:
            image_located=Unisize_image
            break
    return iamge_located #only one image should be output

'''def sobel_locate(iamge,name):
    logging.info('sobel_locate...')
    image_Gauss=Gauss(image)
    iamge_Gary=Gary(image_Gauss)
    image_Sobel=Enhencement(iamge_Gary)
    image_Binary=Binary(image_Sobel)
    image_Close=Close(image_Binary)
    image_Edge,contours=Edge(image_Close)
    
    image_located=segment_box(image,contours,'sobel_')

    return image_located'''


#########################

if __name__ == '__main__':
    ini()
    path='./image_proprocess/'
    
    path_='./image_proprocess/char_seg.jpg'
    
    image_orig=np.array(cv.imread(path_))
    
    
    image_Gauss=Gauss(image_orig)
    save_image(image_Gauss,"image_Gauss")
    
    iamge_Gary=Gary(image_Gauss)
    save_image(iamge_Gary,"image_Gary")
    
    image_Sobel=Enhencement(iamge_Gary)
    save_image(image_Sobel,"image_Sobel")
    
    image_Binary=Binary(image_Sobel)
    save_image(image_Binary,"image_Binary")
    
    image_Close=Close(image_Binary)
    save_image(image_Close,"image_Close")
    for i in range(0,1):
        image_Close=Close(image_Close)
        save_image(image_Close,"image_Close")
    

   
    image_Edge,contours=Edge('./image_proprocess/image_Close.jpg')
    save_image(image_Edge,"image_Edge")
    #print np.array(contours).shape
    filter=1 #1 means open the two filters 
    if filter!=1:
        image_with_MinRect=draw_Min_Rect(image_orig,contours)
        save_image(image_with_MinRect,"image_with_MinRect")
    else:
        contours_update,Box_update=Angle_filter(contours)
        contours_update,Box_update=Rect_filter(contours_update)
        print np.array(contours_update).shape[0]
        image_with_MinRect_update=draw_Min_Rect(image_orig,contours_update)
        save_image(image_with_MinRect_update,"image_with_MinRect_update")
        
    
    
    
    
    
    
    
    


INFO -[:20]- **************************Mason(2018-05-27 15-23-35)*******************
INFO -[:36]- Gauss...
INFO -[:30]- save image:image_Gauss ,done
INFO -[:42]- Gary...
INFO -[:30]- save image:image_Gary ,done
INFO -[:48]- Enhencement...
INFO -[:30]- save image:image_Sobel ,done
INFO -[:58]- Binary...
INFO -[:30]- save image:image_Binary ,done
INFO -[:65]- Close...
INFO -[:30]- save image:image_Close ,done
INFO -[:65]- Close...
INFO -[:30]- save image:image_Close ,done
INFO -[:76]- Edge...
INFO -[:30]- save image:image_Edge ,done
INFO -[:106]- Angle_filter...
INFO -[:121]- Rect_filter...
INFO -[:90]- draw_Min_Rect...
INFO -[:30]- save image:image_with_MinRect_update ,done


0


In [1]:
#decoding: utf-8
#encoding: utf-8

#openCV transform C++ code to python
#python2.7  openCV3.4

#Color_locate

import numpy as np
import cv2 as cv
import os
import logging
import time 

def ini():  
    
    TM=time.strftime("%Y-%m-%d %H-%M-%S",time.localtime())
    LOG_FORMAT = "%(levelname)s -[:%(lineno)d]- %(message)s"
    logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
    logging.info('**************************Mason(%s)*******************'%(TM,))
    
def show(image,name):
    cv.imshow('%s'%name,image)
    cv.waitKey(0)
    logging.info('show image....')
    
def save_image(image,name):
    path_out=path+name+'.jpg'
    cv.imwrite(path_out,image)
    logging.info('save image:%s ,done'%name)




#将RGB模型图像映射到HVS模型
def HSV(image):#image is a equliaed image
    logging.info('HSV...')
    image_HSV= cv.cvtColor(image, cv.COLOR_BGR2HSV)
    H, S, V = cv.split(image_HSV)
    save_image(H,'H')
    save_image(S,'S')
    save_image(V,'V')
    return image_HSV, H, S, V

def Gary(image):
    logging.info('Gary...')
    iamge_Gary=cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return iamge_Gary

#直方图均衡/灰度均衡，使得输入图像在每一级灰度上都有相同的像素点输出,
#输入为单通道，所以对于多通道的RGB要3个通道单独输入
def Equalization(image_RGB):
    logging.info('RGB_Equalization...')
    B, G, R = cv.split(image_RGB)
    #print np.array(R).shape
    save_image(R,'R')
    save_image(G,'G')
    save_image(B,'B')
    
    R_Equal=cv.equalizeHist(R)
    save_image(R_Equal,'R_Equal')
    G_Equal=cv.equalizeHist(G)
    save_image(G_Equal,'G_Equal')
    B_Equal=cv.equalizeHist(B)
    save_image(B_Equal,'B_Equal')
    
    RGB_merged = cv.merge([B,G,R])
    image_Equal_RGB=RGB_merged
    return image_Equal_RGB

#二值化分割
def Segment_Blue(image_orig,image_HSV,H,S,V):#must be HSV 
    logging.info('Segment_Blue...')
    H_Blue_Min=90
    H_Blue_Max=130
    S_Blue_Min=80
    S_Blue_Max=255
    V_Blue_Min=90
    V_Blue_Max=255
    LowerBlue = np.array([H_Blue_Min, S_Blue_Min,V_Blue_Min])
    UpperBlue = np.array([H_Blue_Max, S_Blue_Max,V_Blue_Max])
    mask= cv.inRange(image_HSV, LowerBlue, UpperBlue)#in range then set to 1, 白色； 0=背景黑色
    Things= cv.bitwise_and(image_orig,image_orig,mask=mask)#图像与运算，类似于显示原图中的区域
    image_Segment_Blue=Things
    return image_Segment_Blue


def Segment_Yellow(image_orig,image_HSV,H,S,V):#must be HSV 
    logging.info('Segment_Yellow...')
    H_Yellow_Min=15
    H_Yellow_Max=26
    S_Yellow_Min=43
    S_Yellow_Max=255
    V_Yellow_Min=46
    V_Yellow_Max=255
    LowerYellow = np.array([H_Yellow_Min, S_Yellow_Min,V_Yellow_Min])
    UpperYellow = np.array([H_Yellow_Max, S_Yellow_Max,V_Yellow_Max])
    mask= cv.inRange(image_HSV, LowerYellow, UpperYellow)#in range then set to 1, 白色； 0=背景黑色
    Things= cv.bitwise_and(image_orig,image_orig,mask=mask)#图像与运算，类似于显示原图中的区域
    image_Segment_Yellow=Things
    
    return image_Segment_Yellow
    



#计算边界/轮廓
#func explaination https://blog.csdn.net/hjxu2016/article/details/77833336
def Edge(path):#should input binary image
    logging.info('Edge...')
    #calculation
    image=cv.imread(path,cv.CV_8UC1) 
    binary,contours, hierarchy = cv.findContours(image,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE) 
    #in open CV3， findContours return 3 value; so my version is openCV3

    #draw boundary on the image
    #image=cv.drawContours(image,contours,-1,(0,0,255),3)#-1:draw all edge;  3:line width；（0，0，255):RGB color scaler
    #image_Edge=image
    return contours #outpur contours is a array, 

#不相连的轮廓求最小外界矩形,并添加到原图中
def draw_Min_Rect(image,contours):
    logging.info('draw_Min_Rect...')
    N= np.array(contours).shape[0]
    for i in range (0,N):
        min_=cv.minAreaRect(contours[i])
        box = cv.boxPoints(min_) 
        box = np.int0(box)
        cv.drawContours(image, [box], 0, (0,0,255), 3)
    #cv.imshow('0', image)
    #cv.waitKey(0)# the number of windows
    image_with_MinRect=image
    return image_with_MinRect


#Area筛选box; should after the Angle filter
def Rect_filter(conts):
    logging.info('Rect_filter...')
    N= np.array(conts).shape[0]
    contours_update=[]
    Box_update=[]
    for i in range (0,N):
        Rect=cv.minAreaRect(conts[i])
        Area_=Rect[1][0]*Rect[1][1]
        if int(Area_) >=500 and int(Area_)<=15000 : 
            logging.info('W:%s, H:%s, Ratio:%s'%(Rect[1][0], Rect[1][1],(Rect[1][0])/(Rect[1][1])))
            logging.info( 'Area:%s'%Area_)
            if abs(Rect[2])<=20:
                if (Rect[1][0])/(Rect[1][1])>=2 and (Rect[1][0])/(Rect[1][1])<=5:
                    contours_update.append(conts[i])
                    Box_update.append(Rect)
            else:
                if (Rect[1][1])/(Rect[1][0])>=2 and (Rect[1][1])/(Rect[1][0])<=5:
                    contours_update.append(conts[i])
                    Box_update.append(Rect)
    return contours_update,Box_update

def Angle_filter(conts):
    logging.info('Angle_filter...')
    N= np.array(conts).shape[0]
    Box_update=[]
    contours_update=[]
    for i in range (0,N):
        Rect=cv.minAreaRect(conts[i])
        A=Rect[2]#anglle\
        
        if abs(A)>=80 and abs(A)<=90:
            contours_update.append(conts[i])
            Box_update.append(Rect)
        else:
            if abs(A)<=10:
                contours_update.append(conts[i])
                Box_update.append(Rect)
    return contours_update,Box_update

def color_locate(image):#yellow or blue
    image_HSV=HSV(image)
    image_Equal=Equalization(image_HSV)
    
    #Blue
    image_Segment_Blue=Segment_Blue(image_Equal)
    save_image(image_Segment_Blue,'image_Segment_Blue')
    path=path+"image_Segment_Blue"
    image_Segment_Blue=cv.imread(path)
    image_Edge_b,contours_b=Edge(image_Segment_Blue)
    k_b= np.array(contours_b).shape[0]
    image_located_b=segment_box(image,contours_b,'color_')

    #Yellow
    image_Segment_Yellow=Segment_Yellow(image_Equal)
    image_Edge_y,contours_y=Edge(image_Segment_Yellow)
    k_y=np.array(contours_y).shape[0]
    image_located_y=segment_box(image,contours_y,'color_')

    if k_b==1 or k_b==2:
        iamge_located=image_located_b
        k=k_b
    else:
        iamge_located=image_located_y
        k=k_y
    return  k,iamge_located #k is the number of rectangle





def locate_mechanism(image,name):
    #do color locate first, and judge if need to do sobel locate
    k,result_=color_locate(image)

    if k==1 or k==2:
        image_located=result_
    else:
        image_located=sobel_locate(iamge,name)

    return image_located




#########################

if __name__ == '__main__':
    ini()
    path='./image_proprocess/'
    path_='./image_proprocess/orig.jpg'
    image_orig=cv.imread(path_)
    
    image_Equalized=Equalization(image_orig)
    save_image(image_Equalized,'image_Equalized')
    
    image_HSV,H,S,V=HSV(image_Equalized)
    save_image(image_HSV,'image_HSV')
    #print np.array(image_HSV).shape #is h, [1] is w, [2] is ch
    
    #################
    image_Segment_Blue=Segment_Blue(image_orig,image_HSV,H,S,V)
    save_image(image_Segment_Blue,'image_Segment_Blue') 
    
    contours=Edge(path+'image_Segment_Blue.jpg')
    #os.remove(path+'image_Segment_Blue.jpg')
    

    filter=1
    if filter!=1: #raw model
        image_with_MinRect=draw_Min_Rect(image_orig,contours)
        save_image(image_with_MinRect,'image_with_MinRect')      
    else:
        # filter
        logging.info('filter in ...')
        contours_update,Box_update_A=Angle_filter(contours)
        contours_update,Box_update=Rect_filter(contours_update)
        
        #color check
        if np.array(contours_update).shape[0]!=0: #Blue if 0 hen the car and plate have same color  
            image_with_MinRect_update=draw_Min_Rect(image_orig,contours_update)# draw on the image_orig object  
            save_image(image_with_MinRect_update,'image_with_MinRect_update')
        else:#Yellow
            logging.info('Blue_locate failed, try Yellow')
            image_Segment_Yellow=Segment_Yellow(image_orig,image_HSV,H,S,V)
            save_image(image_Segment_Yellow,'image_Segment_Yellow') 
            contours=Edge(path+'image_Segment_Yellow.jpg')
            #os.remove(path+'image_Segment_Yellow.jpg')

            contours_update,Box_update=Rect_filter(contours)
            image_with_MinRect_update=draw_Min_Rect(image_orig,contours_update)# draw on the image_orig object  
            save_image(image_with_MinRect_update,'image_with_MinRect_update')


INFO -[:20]- **************************Mason(2018-05-27 15-18-03)*******************
INFO -[:53]- RGB_Equalization...
INFO -[:30]- save image:R ,done
INFO -[:30]- save image:G ,done
INFO -[:30]- save image:B ,done
INFO -[:30]- save image:R_Equal ,done
INFO -[:30]- save image:G_Equal ,done
INFO -[:30]- save image:B_Equal ,done
INFO -[:30]- save image:image_Equalized ,done
INFO -[:37]- HSV...
INFO -[:30]- save image:H ,done
INFO -[:30]- save image:S ,done
INFO -[:30]- save image:V ,done
INFO -[:30]- save image:image_HSV ,done
INFO -[:73]- Segment_Blue...
INFO -[:30]- save image:image_Segment_Blue ,done
INFO -[:110]- Edge...
INFO -[:250]- filter in ...
INFO -[:159]- Angle_filter...
INFO -[:138]- Rect_filter...
INFO -[:146]- W:135.0, H:35.0, Ratio:3.85714285714
INFO -[:147]- Area:4725.0
INFO -[:123]- draw_Min_Rect...
INFO -[:30]- save image:image_with_MinRect_update ,done


In [9]:
import cv2  
import numpy as np

def save_image(image,name):
    path_out=path+name+'.jpg'
    c2.imwrite(path_out,image)
path='./image_proprocess/'
path_='./image_proprocess/orig.jpg'
Img=cv2.imread(path_)

HSV = cv2.cvtColor(Img, cv2.COLOR_BGR2HSV)

H, S, V = cv2.split(HSV)
LowerBlue = np.array([90, 80, 90])
UpperBlue = np.array([130, 255, 255])
mask = cv2.inRange(HSV, LowerBlue, UpperBlue)

Things = cv2.bitwise_and(Img,Img,mask=mask)#图像与运算，
path_out=path+'test.jpg'
path_out_=path+'test2.jpg'
cv2.imwrite(path_out,mask)
cv2.imwrite(path_out_,Things)

True